# 使用遷移學習Transfer Learning Sentiment analysis

## (trainable=False)
## Test accuracy: 0.084

## 比 trainable=True 低很多



In [1]:
# 使用:GoogleNews-vectors-negative300.bin
# glove.6B.zip

# code from:
# https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py

In [2]:
'''This script loads pre-trained word embeddings (GloVe embeddings)
into a frozen Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classification of newsgroup messages into 20 different categories).

GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)

20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html
'''

'This script loads pre-trained word embeddings (GloVe embeddings)\ninto a frozen Keras Embedding layer, and uses it to\ntrain a text classification model on the 20 Newsgroup dataset\n(classification of newsgroup messages into 20 different categories).\n\nGloVe embedding data can be found at:\nhttp://nlp.stanford.edu/data/glove.6B.zip\n(source page: http://nlp.stanford.edu/projects/glove/)\n\n20 Newsgroup data can be found at:\nhttp://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html\n'

In [3]:
import fasttext
from pyfasttext import FastText
import time

In [4]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

Using TensorFlow backend.


In [5]:
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, './data/')  #向量檔案位置  glove.6B.100d.txt
TEXT_DATA_DIR = os.path.join(BASE_DIR, './data/20_newsgroup')   #20_newsgroup整包資料夾檔案位置  
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# 把glove.6B.100d.txt檔裡面的向量 與標題單字  切割開來存進字典

In [6]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}  #為字典
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))  #打開檔案
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')   #[1:] 第二位到最後一位給予...   因為第一位是英文單字
    embeddings_index[word] = coefs  #存進去字典
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


# 把資料夾裡面檔案的文章 前處理存進texts

In [7]:
# second, prepare text samples and their labels
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):      
    path = os.path.join(TEXT_DATA_DIR, name)  
    if os.path.isdir(path):
        label_id = len(labels_index)   
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)): 
            if fname.isdigit():              
                fpath = os.path.join(path, fname)                
                if sys.version_info < (3,):
                    f = open(fpath)                
                else:
                    f = open(fpath, encoding='latin-1') 
                  
                t = f.read()                
                i = t.find('\n\n')  # skip header

                if 0 < i:
                    t = t[i:]
                    
                texts.append(t)           
                f.close()
                labels.append(label_id)
print('Found %s texts.' % len(texts))

Processing text dataset
Found 19997 texts.


# 將texts的文章轉為2值化序列  整理好格式之後切割資料集

In [8]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

Found 174074 unique tokens.
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


# 預先訓練好的詞向量在這裡讀(前面整理過的glove.6B.100d.txt 字典)

In [9]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


# CNN 卷積層
## (weights加進前面glove.6B.100d.txt 整理的embedding_matrix)
## 載入預訓練模型和訓練模型

In [10]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()

Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         2000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 128)           82048

In [11]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 207s 13ms/step - loss: 3.6895 - acc: 0.0586 - val_loss: 3.3384 - val_acc: 0.0528
Epoch 2/10
15998/15998 [==============================] - 220s 14ms/step - loss: 3.0759 - acc: 0.0557 - val_loss: 3.0174 - val_acc: 0.0523
Epoch 3/10
15998/15998 [==============================] - 221s 14ms/step - loss: 3.0202 - acc: 0.0530 - val_loss: 2.9934 - val_acc: 0.0508
Epoch 4/10
15998/15998 [==============================] - 235s 15ms/step - loss: 3.0369 - acc: 0.0641 - val_loss: 2.9483 - val_acc: 0.0700
Epoch 5/10
15998/15998 [==============================] - 228s 14ms/step - loss: 2.9658 - acc: 0.0664 - val_loss: 2.9769 - val_acc: 0.0698
Epoch 6/10
15998/15998 [==============================] - 200s 12ms/step - loss: 2.9955 - acc: 0.0757 - val_loss: 2.9526 - val_acc: 0.0803
Epoch 7/10
15998/15998 [==============================] - 246s 15ms/step - loss: 2.9980 - acc: 0.0774 - val_loss: 2.9

In [12]:
score = model.evaluate(x_val, y_val, verbose=0)

In [13]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 2.79599563990214
Test accuracy: 0.0842710677594893
